<a href="https://colab.research.google.com/github/ChabiMax/author_network/blob/main/Code/Scraping%20propre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center><strong><font color="chillipepper">Réseau Wikipédia des auteurs francophones</font></strong></center></h1>
<h5><center><font color="green">Science des réseaux sociaux et économiques <h3><center>


<center>
Maxime Chabriel, Yasmine Houri, Mathis Sansu  

ENSAE 2022
</center>

<h2><center><strong><font color="chillipepper">Scraping des données</font></strong></center></h2>

# Packages

In [90]:
from tqdm import *
import time
import requests
import bs4
from collections import Counter
import json
import os
from google.colab import files
import pickle as pickle
import pandas as pd

# Scraping

In [91]:
raw_data = {} # Dictionnaire pour stocker les données

In [92]:
# URL de la page initiale de notre liste d'auteurs
URL = 'https://fr.wikipedia.org/wiki/Liste_d%27%C3%A9crivains_de_langue_fran%C3%A7aise_par_ordre_chronologique'
response = requests.get(URL) # Go to the page
soup = bs4.BeautifulSoup(response.text, 'html.parser') # Récupération du code html

In [93]:
dictionary = {} # Déclaration d'un dictionnaire vide
# On initialise des attributs
date_naissance = None
date_mort = None
name = 'init'
author_link = 'init'
id = 0
serie = 0
links = soup.find(id="bodyContent").select('a') # Récupération des hyperliens

In [94]:
# Extraction de la liste d'auteurs

for link in links: # Boucle sur les hyperliens récupérés
  string = link.get_text()
  if link.get('href') != None:
    if not any(char.isdigit() for char in string) : 
      if serie == 2 :
        serie = 0
        date_mort = None
        dictionary[id] = {'name' : name, 'link' : author_link, 'date_mort' : date_mort, 'date_naissance' : date_naissance}
      serie = serie+1
      id = id+1
      name = string
      date_naissance = None
      date_mort = None
      if 'https://' in link.get('href'):
        author_link = link.get('href')
      else:
        author_link = 'https://fr.wikipedia.org' + link.get('href')
    else :
      if serie == 1 :
        serie = serie+1
        date_naissance = string
      else :
        if serie == 2 :
          serie = 0
          date_mort = string
          dictionary[id] = {'name' : name, 'link' : author_link, 'date_mort' : date_mort, 'date_naissance' : date_naissance}

In [95]:
# Nettoyage manuel : certains lien récupérés ne correpondent pas à des auteurs
unwanted_list = [key for key in dictionary.keys() if 'modifier' in dictionary[key]['name']]
unwanted_list += [key for key in dictionary.keys() if 'siècle' in dictionary[key]['name']]
unwanted_list += [key for key in dictionary.keys() if 'ms.' in dictionary[key]['name']]
unwanted_list += [key for key in dictionary.keys() if 'Portail' in dictionary[key]['name']]
unwanted_list += [key for key in dictionary.keys() if "Liste d'écrivains de langue française" in dictionary[key]['name']]
unwanted_list += [key for key in dictionary.keys() if "↑" in dictionary[key]['name']]
unwanted_list += [key for key in dictionary.keys() if "" == dictionary[key]['name']]
unwanted_list += [key for key in dictionary.keys() if "Langue française et francophonie" in dictionary[key]['name']]
unwanted_list += [key for key in dictionary.keys() if "Chronologie de la littérature" in dictionary[key]['name']]
# Les commandes au-dessus permettent de récupérer les clés problématiques dans le dictionnaire
# On enlève donc les éléments associés à ces clés dans le dictionnaire
dictionary = dict((key, dictionary[key]) for key in dictionary.keys() if key not in unwanted_list)

In [96]:
 # Extraction de la liste des genres littéraires (mêmes procédures)

URL = 'https://fr.wikipedia.org/wiki/Genre_litt%C3%A9raire'
response = requests.get(URL)
soup = bs4.BeautifulSoup(response.text, 'html.parser')

dictionary_genres = {}
name = 'init'
genre_link = 'init'
id = 0
links = soup.find(id="bodyContent").select('a')
serie = 0
for link in links:
  string = link.get_text()
  if link.get('href') != None:
    id = id+1
    name = string
    if 'https://' in link.get('href'):
      genre_link = link.get('href')
    else:
      genre_link = 'https://fr.wikipedia.org' + link.get('href')
    dictionary_genres[id] = {'name' : name, 'link' : genre_link}


In [97]:
# Nettoyage manuel
unwanted_list = list(range(0,49))
unwanted_list += list(range(278,344))
unwanted_list += [key for key in dictionary_genres.keys() if 'modifier' in dictionary_genres[key]['name']]
unwanted_list += [key for key in dictionary_genres.keys() if 'Portail' in dictionary_genres[key]['name']]
unwanted_list += [key for key in dictionary_genres.keys() if 'Li Shangyin' in dictionary_genres[key]['name']]
unwanted_list += [key for key in dictionary_genres.keys() if 'Sei Shōnagon' in dictionary_genres[key]['name']]
unwanted_list += [key for key in dictionary_genres.keys() if 'Yoshida Kenkō' in dictionary_genres[key]['name']]
unwanted_list += [key for key in dictionary_genres.keys() if 'Pascal Quignard' in dictionary_genres[key]['name']]
unwanted_list += [key for key in dictionary_genres.keys() if 'Sévigné' in dictionary_genres[key]['name']]
unwanted_list += [key for key in dictionary_genres.keys() if 'Caractères' in dictionary_genres[key]['name']]
unwanted_list += [key for key in dictionary_genres.keys() if 'Bruyère' in dictionary_genres[key]['name']]
unwanted_list += [key for key in dictionary_genres.keys() if 'Montandon' in dictionary_genres[key]['name']]
unwanted_list += [key for key in dictionary_genres.keys() if dictionary_genres[key]['link'] in [dictionary[key_bis]['link'] for key_bis in dictionary.keys()]]
unwanted_list += [key for key in dictionary_genres.keys() if any(char.isdigit() for char in dictionary_genres[key]['name'])]
unwanted_list += list(range(285,len(dictionary_genres.keys())))
dictionary_genres = dict((key, dictionary_genres[key]) for key in dictionary_genres.keys() if key not in unwanted_list)

In [98]:
# Reset des keys 
old_dictionary = dictionary.copy()
dictionary = {}
id = 0
for key in old_dictionary :
  dictionary[id] = old_dictionary[key]
  id += 1

old_dictionary_genres = dictionary_genres.copy()
dictionary_genres = {}
for key in old_dictionary_genres :
  dictionary_genres[id] = old_dictionary_genres[key]
  id += 1

In [99]:
# Fonction réduisant une balise html de lien sous forme d'un lien wikipédia classique
def html_to_link(html_link) :
  if html_link.get('href') != None :
    if 'https://' in html_link.get('href'):
      html_link = html_link.get('href')
    else:
      html_link = 'https://fr.wikipedia.org' + html_link.get('href')
  else :
    html_link = ''
  return(html_link)

In [100]:
# Création des listes des liens associés à nos pages Wikipédia (auteurs et genres)
author_links = [dictionary[key]['link'] for key in dictionary.keys()]
genre_links = [dictionary_genres[key]['link'] for key in dictionary_genres.keys()]

In [101]:
# Parcours des pages et création du réseau
for id in tqdm_notebook(dictionary.keys()) :
  authtor_link = dictionary[id]['link']
  response = requests.get(authtor_link)
  soup = bs4.BeautifulSoup(response.text, 'html.parser')
  links = soup.find(id="bodyContent").select('a')
  network_aslink = []
  net_genre_aslink = []
  for link in links:
    link = html_to_link(link)
    if link in author_links :
      network_aslink.append(link)
    if link in genre_links :
      net_genre_aslink.append(link)
  network = []
  net_genre = []
  for link in network_aslink :
    for id_bis in dictionary.keys() :
      if link == dictionary[id_bis]['link'] :
        network.append(id_bis)
  for link in net_genre_aslink :
    for id_bis in dictionary_genres.keys() :
      if link == dictionary_genres[id_bis]['link'] :
        net_genre.append(id_bis)
  counted_network = Counter(network)
  counted_net_genre = Counter(net_genre)
  dictionary[id]['taille_html_article'] = len(str(soup))
  dictionary[id]['network_interauteur'] = list(counted_network.keys())
  dictionary[id]['poids_liens_interauteur'] = list(counted_network.values())
  dictionary[id]['genres_mentionnes'] = list(counted_net_genre.keys())


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/1584 [00:00<?, ?it/s]

# Formatage données

In [102]:
raw_data1 = dictionary.copy()
for key in raw_data1.keys() :
  raw_data1[key]['type'] = 'auteur'

for key in dictionary_genres.keys() :
  raw_data1[key] = dictionary_genres[key].copy()
  raw_data1[key]['type'] = 'genre'

for key_genre in raw_data1.keys() :
  if raw_data1[key_genre]['type'] == 'genre' :
    raw_data1[key_genre]['auteurs_du_genre'] = []
for key_auteur in raw_data1.keys() :
  if raw_data1[key_auteur]['type'] == 'auteur' :
    for genre in raw_data1[key_auteur]['genres_mentionnes'] :
      raw_data1[genre]['auteurs_du_genre'] = raw_data1[genre]['auteurs_du_genre'] + [key_auteur]

for key_genre in raw_data1.keys() :
  if raw_data1[key_genre]['type'] == 'auteur' :
    try :
      raw_data1[key_genre]['date_naissance'] = int(raw_data1[key_genre]['date_naissance'])
    except :
      raw_data1[key_genre]['date_naissance'] = None
    try :
      raw_data1[key_genre]['date_mort'] = int(raw_data1[key_genre]['date_mort'])
    except :
      raw_data1[key_genre]['date_mort'] = None

for key_genre in raw_data1.keys() :
  if raw_data1[key_genre]['type'] == 'genre' :
    raw_data1[key_genre]['debut_genre'] = min([raw_data1[auteur]['date_naissance'] for auteur in raw_data1 if raw_data1[auteur]['type'] == 'auteur' and key_genre in raw_data1[auteur]['genres_mentionnes']] + [9999])
    raw_data1[key_genre]['fin_genre'] = max([raw_data1[auteur]['date_mort'] for auteur in raw_data1 if raw_data1[auteur]['type'] == 'auteur' and key_genre in raw_data1[auteur]['genres_mentionnes']] + [0])

for key_auteur1 in tqdm_notebook(raw_data1.keys()) :
  if raw_data1[key_auteur1]['type'] == 'auteur' :
    raw_data1[key_auteur1]['contemporain'] = []
    raw_data1[key_auteur1]['poids_contempo_diff_naissance'] = []
    raw_data1[key_auteur1]['poids_contempo_annees_communes'] = []
    for key_auteur2 in raw_data1.keys() :
      if (raw_data1[key_auteur2]['type'] == 'auteur') & (key_auteur2 != key_auteur1) :
        mort_1 = [raw_data1[key_auteur1]['date_mort'] if raw_data1[key_auteur1]['date_mort'] != None else 2022 if raw_data1[key_auteur1]['date_naissance'] > 1900 else None]
        mort_2 = [raw_data1[key_auteur2]['date_mort'] if raw_data1[key_auteur2]['date_mort'] != None else 2022 if raw_data1[key_auteur2]['date_naissance'] > 1900 else None]
        naissance_1 = [raw_data1[key_auteur1]['date_naissance'] if raw_data1[id]['date_mort'] != None else None]
        naissance_2 = [raw_data1[key_auteur2]['date_naissance'] if raw_data1[key_auteur2]['date_mort'] != None else None]
        try:
          annees_1 = set(range(naissance_1[0],mort_1[0]))
          annees_2 = range(naissance_2[0],mort_2[0])
          annees_communes = list(annees_1.intersection(annees_2))
          if (mort_1[0] > naissance_2[0] and naissance_1[0] < mort_2[0]) or (mort_2[0] > naissance_1[0] and naissance_2[0] < mort_1[0]):
            raw_data1[key_auteur1]['contemporain'].append(key_auteur2)
            raw_data1[key_auteur1]['poids_contempo_diff_naissance'].append(abs(naissance_1[0] - naissance_2[0]))
            raw_data1[key_auteur1]['poids_contempo_annees_communes'].append(len(annees_communes))

        except :
          pass

In [105]:
df = pd.DataFrame(raw_data1).T
df.tail(10)

,name,link,date_mort,date_naissance,taille_html_article,network_interauteur,poids_liens_interauteur,genres_mentionnes,type,contemporain,poids_contempo_diff_naissance,poids_contempo_annees_communes,auteurs_du_genre,debut_genre,fin_genre
1782,formulette d'élimination,https://fr.wikipedia.org/wiki/Formulette_d%27%...,NaN,NaN,NaN,NaN,NaN,NaN,genre,NaN,NaN,NaN,[],9999,0
1783,haïku,https://fr.wikipedia.org/wiki/Ha%C3%AFku,NaN,NaN,NaN,NaN,NaN,NaN,genre,NaN,NaN,NaN,"[1243, 1273, 1445]",1953,9999
1784,virelangue,https://fr.wikipedia.org/wiki/Virelangue,NaN,NaN,NaN,NaN,NaN,NaN,genre,NaN,NaN,NaN,[746],1898,1990
1785,Li Shangyin,https://fr.wikipedia.org/wiki/Li_Shangyin,NaN,NaN,NaN,NaN,NaN,NaN,genre,NaN,NaN,NaN,[],9999,0
1786,Sei Shōnagon,https://fr.wikipedia.org/wiki/Sei_Sh%C5%8Dnagon,NaN,NaN,NaN,NaN,NaN,NaN,genre,NaN,NaN,NaN,[],9999,0
1787,Yoshida Kenkō,https://fr.wikipedia.org/wiki/Yoshida_Kenk%C5%8D,NaN,NaN,NaN,NaN,NaN,NaN,genre,NaN,NaN,NaN,[],9999,0
1788,Contrainte,https://fr.wikipedia.org/wiki/Litt%C3%A9rature...,NaN,NaN,NaN,NaN,NaN,NaN,genre,NaN,NaN,NaN,[],9999,0
1789,Expérimentale,https://fr.wikipedia.org/wiki/Litt%C3%A9rature...,NaN,NaN,NaN,NaN,NaN,NaN,genre,NaN,NaN,NaN,[1493],1973,9999
1790,Littérature numérique,https://fr.wikipedia.org/wiki/Litt%C3%A9rature...,NaN,NaN,NaN,NaN,NaN,NaN,genre,NaN,NaN,NaN,[],9999,0
1791,Fanfiction,https://fr.wikipedia.org/wiki/Fanfiction,NaN,NaN,NaN,NaN,NaN,NaN,genre,NaN,NaN,NaN,[],9999,0


# Import des données

In [104]:
# Import sous forme pickle (objet python assez léger)
import pickle
with open('raw_data.pkl', 'wb') as f:
    pickle.dump(raw_data1, f)
files.download('raw_data.pkl')